<h1 style="color: skyblue"> Deeptech Engineer Intern Challenge </h1>

In [1]:
# imports
import pandas as pd
from fuzzywuzzy import process

/home/marius/.local/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [19]:
# Facebook dataset
chunk_size = 1000000 
facebook_dataset = pd.read_csv('facebook_dataset.csv', delimiter=',', quotechar='"', on_bad_lines='skip')
facebook_dataset.head(5)

,domain,address,categories,city,country_code,country_name,description,email,link,name,page_type,phone,phone_country_code,region_code,region_name,zip_code
0,euro-hygiene-34.fr,"134 rue entrepreneurs, za du vigné, 30420, cal...",NaN,calvisson,fr,france,NaN,NaN,https://euro-hygiene-34.fr,Euro Hygiène,LocalBusiness,NaN,NaN,occ,occitanie,30420
1,lakesidehomeservices.ca,NaN,Appliance Repair & Maintenance|Home Builders &...,NaN,ca,NaN,NaN,NaN,https://lakesidehomeservices.ca,Lakeside Home Services,LocalBusiness,1.613832e+10,ca,NaN,NaN,NaN
2,rossiterboats.com,NaN,Boats & Yachts Dealers|Boat Tours & Cruises,NaN,NaN,NaN,At Rossiter Boats we build semi-custom power b...,info@rossiterboats.com,http://rossiterboats.com,Rossiter Boats,LocalBusiness,NaN,NaN,NaN,NaN,NaN
3,caa-architecture.com,"601 cordova st w # 270, v6b 1g1, vancouver, bc...",Architects & Architectural Services|Other Engi...,vancouver,ca,canada,NaN,office@caa-arcitecture.com,http://caa-architecture.com,Chandler Associates Architecture Inc.,LocalBusiness,1.604687e+10,ca,bc,british columbia,v6b 1g1
4,apexsurety.ca,"unit 3 - 4 donald street, r3l 2t7, winnipeg, m...",NaN,winnipeg,ca,canada,NaN,NaN,https://apexsurety.ca,Apex Surety & Insurance Ltd.,LocalBusiness,1.204560e+10,ca,mb,manitoba,r3l 2t7


In [3]:
# Google dataset

google_dataset = pd.read_csv('google_dataset.csv', delimiter=',', quotechar='"', on_bad_lines='skip')
google_dataset.head(5)

/tmp/ipykernel_7018/98803827.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  google_dataset = pd.read_csv('google_dataset.csv', delimiter=',', quotechar='"', on_bad_lines='skip')


,address,category,city,country_code,country_name,name,phone,phone_country_code,raw_address,raw_phone,region_code,region_name,text,zip_code,domain
0,"28 Central Coast Hwy, West Gosford NSW 2250, A...",Fabric-Based Home Goods,gosford,au,australia,Spotlight West Gosford,+61243355946,au,"West Gosford NSW, Australia",+61 2 4335 5946,nsw,new south wales,"4.1 (766) · Craft store West Gosford NSW, Aust...",2250,spotlightstores.com
1,"400 Scott St, St. Catharines, ON L2M 3W2, Canada",Book Stores,st. catharines,ca,canada,Heritage Christian Book Store,+19059374553,ca,400 Scott St · In Grantham Plaza,+1 905-937-4553,on,ontario,4.7 (100) · Book store 400 Scott St · In Grant...,l2m 3w2,bookmanager.com
2,"191 Pleasant St, Yarmouth, NS B5A 2J9, Canada",Other Building Material Retailers,yarmouth,ca,canada,Pleasant Timber Mart,+19027429181,ca,"7+ years in business · Yarmouth, NS, Canada",+1 902-742-9181,ns,nova scotia,4.7 (40) · Building materials store 7+ years i...,b5a 2j9,timbermart.ca
3,"11040 Santa Monica Blvd Suite 370, Los Angeles...",Plastic Surgery Clinics,los angeles,us,united states,Skin Specifics Medical Spa West LA,+18184268353,us,7+ years in business · 11040 Santa Monica Blvd...,+1 818-426-8353,ca,california,4.3 (15) · Medical spa 7+ years in business · ...,90025,linktr.ee
4,NaN,Catering & Delivery,NaN,NaN,NaN,Lakay express,NaN,NaN,NaN,NaN,NaN,NaN,5.0 (1) · Mobile caterer Open ⋅ Closes 9PM,NaN,linktr.ee


In [4]:
# Website dataset

website_dataset = pd.read_csv('website_dataset.csv', delimiter=';', quotechar='"', on_bad_lines='skip')
website_dataset.head(5)

,root_domain,domain_suffix,language,legal_name,main_city,main_country,main_region,phone,site_name,tld,s_category
0,converterman.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,clothesencounter.ca,ca,en,NaN,cardigan,canada,prince edward island,13066937766,Clothes Encounter,ca,Shoes & Other Footwear Stores
2,investa.com.au,com.au,en,Investa Wholesale Funds Management Limited,brisbane,australia,queensland,61282269300,Investa Property Group,au,Real Estate Developers
3,timminsgarage.com,com,en,Timmins Garage Inc.,timmins,canada,ontario,18775896640,Timmins Garage,com,Automobile Dealers & Manufacturers
4,destinate.ca,ca,en,Destinate Group Ltd.,NaN,Canada,NaN,NaN,Destinate Group,ca,Business Consulting


In [5]:
# common columns

columns_facebook = set(facebook_dataset.columns)
columns_google = set(google_dataset.columns)
columns_website = set(website_dataset.columns)


# function to find similar columns
def find_similar_columns(columns1, columns2, columns3):
    similar_columns = []
    for col1 in columns1:
        # find the most similar column in columns2 and columns3
        match2 = process.extractOne(col1, columns2)
        match3 = process.extractOne(col1, columns3)
        # consider a match if similarity score is above a threshold (80)
        if match2[1] > 80 and match3[1] > 80:
            similar_columns.append((col1, match2[0], match3[0]))
    return similar_columns

# find similar columns across the datasets
similar_columns = find_similar_columns(columns_facebook, columns_google, columns_website)

# print the similar columns
for col1, col2, col3 in similar_columns:
    print(f"Similar Columns: {col1}, {col2}, {col3}")

Similar Columns: phone, phone, phone
Similar Columns: phone_country_code, phone_country_code, phone
Similar Columns: domain, domain, root_domain
Similar Columns: name, name, legal_name
Similar Columns: city, city, main_city


In [6]:
# modify root_domain with doman in website dataset
website_dataset.rename(columns = {'root_domain': 'domain'}, inplace = True)
website_dataset.head(5)

,domain,domain_suffix,language,legal_name,main_city,main_country,main_region,phone,site_name,tld,s_category
0,converterman.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,clothesencounter.ca,ca,en,NaN,cardigan,canada,prince edward island,13066937766,Clothes Encounter,ca,Shoes & Other Footwear Stores
2,investa.com.au,com.au,en,Investa Wholesale Funds Management Limited,brisbane,australia,queensland,61282269300,Investa Property Group,au,Real Estate Developers
3,timminsgarage.com,com,en,Timmins Garage Inc.,timmins,canada,ontario,18775896640,Timmins Garage,com,Automobile Dealers & Manufacturers
4,destinate.ca,ca,en,Destinate Group Ltd.,NaN,Canada,NaN,NaN,Destinate Group,ca,Business Consulting


In [7]:
# make a new DataFrame

merged_df = pd.merge(facebook_dataset['domain'], google_dataset['domain'], on = 'domain', how = 'outer')
merged_df = pd.merge(merged_df, website_dataset['domain'], on = 'domain', how = 'outer')

# drop duplicates
merged_df.drop_duplicates(inplace = True)

# make domain as primary key / id
merged_df.set_index('domain', inplace = True)
merged_df.sample(5)

""
domain
centralcityroofing.ca
electromedia.ca
kootenaimoon.com
enidpublicschools.org
loneoakbrew.com


In [8]:
# add to dataframe following common columns: Category, Address, Phone, Company names.

# first preprocess the categories to have the same column name

# category column name
facebook_dataset.rename(columns = {'categories': 'category'}, inplace = True)
website_dataset.rename(columns = {'s_category': 'category'}, inplace = True)

In [9]:
facebook_dataset['category'].sample(5)

51098                                      NaN
57985                                      NaN
60569       Sports Medicine & Physical Therapy
45156    Towing Wrecking & Roadside Assistance
63639                                      NaN
Name: category, dtype: object

In [10]:
website_dataset['category'].sample(5)

49361                        Cybersecurity
65570      Agriculture & Farm Associations
9632               Colleges & Universities
19142                          Book Stores
64833    Security Guards & Patrol Services
Name: category, dtype: object

In [11]:
# address is missing from website dataset

# phone is the same

# website dataset is different, should be name
website_dataset.rename(columns = {'legal_name': 'name'}, inplace = True)
website_dataset['name'].sample(5)

36302                     NaN
53915                     NaN
41286                     NaN
36495                     NaN
19960    DKM Engineering Ltd.
Name: name, dtype: object

In [12]:
# drop duplicates from all datasets
website_dataset.drop_duplicates(inplace = True)
facebook_dataset.drop_duplicates(inplace = True)
google_dataset.drop_duplicates(inplace = True)

In [13]:
# merge category column
merged_category = pd.merge(facebook_dataset[['domain', 'category']], google_dataset[['domain', 'category']], on = "domain")
merged_category = pd.merge(merged_category, website_dataset[['domain', 'category']], on = "domain")
merged_category.rename(columns = {'category': 'category_z'}, inplace = True)
merged_category.drop_duplicates(inplace  = True)
merged_category.sample(5)

,domain,category_x,category_y,category_z
17265,securespecialized.com,Business Consulting| Carriers & Brokers,NaN,Limousines
183787,sarniatopgearauto.com,Wheels & Tire Dealers,Auto Parts & Repair Shops,Auto Services
6983,pschev.ca,Automobile Dealers & Manufacturers,NaN,Automobile Dealers & Manufacturers
187014,hardblock.com,Industrial Machinery & Supplies,Other Building Material Retailers,Asphalt & Paving Products
161251,aliceroofing.ca,Home Builders & Renovation Contractors|Roofing...,Roofing Contractors,Roofing Contractors


In [14]:
# merge address column
merged_address = pd.merge(facebook_dataset[['domain', 'address']], google_dataset[['domain', 'address']], on = "domain")
merged_address.rename(columns = {'address': 'address_z'}, inplace = True)
merged_address.drop_duplicates(inplace  = True)
merged_address.sample(5)

,domain,address_x,address_y
114533,destinyroofing.ca,"40 regan road unit#7, l7a 1a7, brampton, on, c...","19 Kenview Blvd unit 34, Brampton, ON L6T 5G6,..."
82048,marcottedesign.com,"376 shaw st., m6j2x3, toronto, on, canada, ont...","376 Shaw St, Toronto, ON M6J 2X3, Canada"
238465,facebook.com,NaN,"39741 Pitkin Rd, Paonia, CO 81428"
185979,northpower.com,"28 mt pleasant road, 0110, whangarei, new zeal...","Northpower - Warkworth 5 Gumfield Drive, Warkw..."
82690,ozguntr.com,"gayrettepe mah. gönenoğlu sk. no:7 beşiktaş, 3...","Gayrettepe, Gönenoğlu Sk. No:7, 34349 Beşiktaş..."


In [15]:
# merge category and address into a new dataframe
DF = pd.merge(merged_category, merged_address, on = 'domain')
DF.drop_duplicates(inplace = True)
DF.head(5)

,domain,category_x,category_y,category_z,address_x,address_y
0,euro-hygiene-34.fr,NaN,Pest Services & Agricultural Chemicals,Pest Services & Agricultural Chemicals,"134 rue entrepreneurs, za du vigné, 30420, cal...","3+ years in business · Calvisson, France"
1,lakesidehomeservices.ca,Appliance Repair & Maintenance|Home Builders &...,Appliance Repair & Maintenance,Appliance Repair & Maintenance,NaN,NaN
2,rossiterboats.com,Boats & Yachts Dealers|Boat Tours & Cruises,Boats & Yachts Dealers,Boats & Yachts Repair Shops,NaN,"Markdale, ON, Canada"
3,caa-architecture.com,Architects & Architectural Services|Other Engi...,Architects & Architectural Services,Architects & Architectural Services,"601 cordova st w # 270, v6b 1g1, vancouver, bc...","The Marine Building, 355 Burrard St Suite 180,..."
4,apexsurety.ca,NaN,"Insurance - Agents, Carriers & Brokers",Liability Insurance Agency,"unit 3 - 4 donald street, r3l 2t7, winnipeg, m...","4 Donald St #3, Winnipeg, MB R3L 2T7, Canada"


In [20]:
# merge phone column
merged_phone = pd.merge(facebook_dataset[['domain', 'phone']], google_dataset[['domain', 'phone']], on = "domain")
merged_phone = pd.merge(merged_phone, website_dataset[['domain', 'phone']], on = "domain")
merged_phone.rename(columns = {'phone': 'phone_z'}, inplace = True)
merged_phone.drop_duplicates(inplace  = True)
merged_phone.sample(5)

,domain,phone_x,phone_y,phone_z
15980,encorepaintingltd.com,1.416671e+10,14166706589.0,14166706589
246615,facebook.com,NaN,14506577232.0,18772765554
46603,masthair.com,NaN,+17809086278,NaN
98748,gauldnurseries.com,1.905354e+10,+19053542392,19053542392
305916,farmingdale.edu,1.631420e+10,16314202111.0,19344202000


In [22]:
# merge DF and phone into a new dataframe
# DF = pd.merge(DF, merged_phone, on = 'domain')
# DF.head(5)

In [23]:
# merge company column
merged_company = pd.merge(facebook_dataset[['domain', 'name']], google_dataset[['domain', 'name']], on = "domain")
merged_company = pd.merge(merged_company, website_dataset[['domain', 'name']], on = "domain")
merged_company.rename(columns = {'name': 'name_z'}, inplace = True)
merged_company.drop_duplicates(inplace  = True)
merged_company.sample(5)

,domain,name_x,name_y,name_z
283028,greenacreservices.ca,Green Acre Services,Green Acre Services,NaN
40911,stjohn.org.nz,St John New Zealand,St John Ambulance,NaN
269167,kindercare.com,Mother Goose School,South Hulen KinderCare,KinderCare Learning Centers LLC
285002,hilton.com,Hilton Newsroom,Hilton Garden Inn Boulder,Hilton Domestic Operating Company Inc.
232947,facebook.com,Facebook,Solunqua Orchards,Facebook Ireland Ltd.


In [24]:
# merge DF and name into a new dataframe
DF = pd.merge(DF, merged_company, on = 'domain')
DF.head(5)

MemoryError: Unable to allocate 12.4 TiB for an array with shape (1701613050614,) and data type int64